4-1 計算環境:  Google Colaboratory

メリット
    - 環境構築に手間がかからない  
    - ライブラリが入っている
    - GPU計算が簡単
    

利用するライブラリ

- transformaers  
    ニューラル言語モデルのライブラリ  
- Fugashi  
    MeCabのPython用ライブラリ  
- ipadic  
    MeCabで形態素解析を行う際に用いる辞書  
    
Colaboratory では, !の後にコマンドを書くとコマンド実行できる





In [2]:
import torch 
from transformers import BertJapaneseTokenizer, BertModel


- Transformers  
    - Huggingface 社が提供しているN言語モデルのOSS  
    - cl-touhoku/bert-base-japanese-whole-word-masking  
        - 東北大学がWikipediaの日本語記事データを用いて学習したモデル  
        
    - 処理手順  
    1. トークナイザを用いて,文章をトークン化
    2. 処理したデータをBERTに入力し,出力を得る  
    
- トークナイザ  
    文章をトークンに分割し,BERTに入力できる形に変換.

In [5]:
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
# 学習済みのトークナイザをロード
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

内部では, 
1. MeCabで単語分割  
2. WOrdPieceでトークン化  
している.

In [9]:
#トークン化してみる
tokenizer.tokenize('明日は自然言語処理の勉強をしよう.')

['明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う', '.']

In [8]:
tokenizer.tokenize('明日はマシンラーニングの勉強をしよう.')

['明日', 'は', 'マシン', '##ラー', '##ニング', 'の', '勉強', 'を', 'しよ', 'う', '.']

｢##｣はサブワード分割の結果,最初の単語以外につく.

In [12]:
tokenizer.tokenize('機械学習を中国語にすると机器学习だ.')

['機械', '学習', 'を', '中国', '語', 'に', 'する', 'と', '机', '器', '学', '[UNK]', 'だ', '.']

习が未知語[UNK]トークンに変換されている.

In [14]:
#文章を符号化(ID化)するには encode()を使う.
input_ids = tokenizer.encode('明日は自然言語処理の勉強をしよう.')
print(input_ids)


[2, 11475, 9, 1757, 1882, 2762, 5, 8192, 11, 2132, 205, 143, 3]


encode()は,  
- トークン列の先頭に[CLS]  
- 末尾に[SEP]  
を自動で追加する.


In [16]:
# ID ー> トークン は convert_ids_to_tokens()
tokenizer.convert_ids_to_tokens(input_ids)


['[CLS]', '明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う', '.', '[SEP]']

In [21]:
#BERTを利用するためにトークン列の長さ(系列長)を同じに揃える
#系列長が短ければ特殊トークン[PAD]を末尾に足す
#ながければ末尾トークンを削る
#[PAD]部分は処理に関係ないので,
#関係ある部分のみを表すattenntion_maskを用意しておく

# tokenizerで指定した系列帳に揃えられたidやmaskの辞書が手に入る
# max_length : 系列長
# padding='max_length': 足りなかったら12まで[PAD]を足す
# truncation: 多かったら削る

text = '明日の天気は晴れだ.'
encoding = tokenizer(
    text, max_length=12, padding='max_length', truncation=True
)
print('# encoding:')
print(encoding)

tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
print('# tokens:')
print(tokens)

# encoding:
{'input_ids': [2, 11475, 5, 11385, 9, 16577, 75, 143, 3, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]}
# tokens:
['[CLS]', '明日', 'の', '天気', 'は', '晴れ', 'だ', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]']


In [22]:
#系列長を短くしてみる
encoding = tokenizer(
    text, max_length=6, padding='max_length', truncation=True
)
print('# encoding:')
print(encoding)

tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
print('# tokens:')
print(tokens)

# encoding:
{'input_ids': [2, 11475, 5, 11385, 9, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}
# tokens:
['[CLS]', '明日', 'の', '天気', 'は', '[SEP]']


In [24]:
#truncationをFalseにしてみる
encoding = tokenizer(
    text, max_length=6, padding='max_length', truncation=False
)
print('# encoding:')
print(encoding)

tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
print('# tokens:')
print(tokens)

# encoding:
{'input_ids': [2, 11475, 5, 11385, 9, 16577, 75, 143, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
# tokens:
['[CLS]', '明日', 'の', '天気', 'は', '晴れ', 'だ', '.', '[SEP]']


In [29]:
#復数の文章をリストで渡せる
text_list = [ '明日の天気は晴れだ.'
             ,'パソコンが急に動かなくなった.' ]

#それぞれリストが返ってくる
# padding=longest  で最も長い文章が系列長になる

# BERTに入力する場合は数値配列はPyTorchの多次元配列を扱うための型
# torch.Tensorに変換する必要がある
# return_tensors='pt'でテンソルとして出力されそのまま入力可能
encoding = tokenizer( text_list
                     , max_length = 10
                     , padding='max_length'
                     , truncation=True
                     , return_tensors='pt'
)
print('# encoding:')
print(encoding)

# encoding:
{'input_ids': tensor([[    2, 11475,     5, 11385,     9, 16577,    75,   143,     3,     0],
        [    2,  6311,    14,  1132,     7, 16084,   332,    58,    10,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [34]:
#符号化(トークン化→ID化)されたデータをBERTに入力する
#モデルのロード
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
bert = BertModel.from_pretrained(model_name)

torch.device('mps')

#BERTをGPUに載せる
bert = bert.cuda()

#モデルの概要
print(bert.congif)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: Expected one of cpu, cuda, xpu, mkldnn, opengl, opencl, ideep, hip, msnpu, xla, vulkan device type at start of device string: mps